# Hyperparameter Tuning

모델 깊이: 1 ~ 5, 모델 너비: 32 ~ 512, dropuout: 0.1 ~ 0.5, learning_rate: 1e-5 ~ 1e-2

In [1]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.4 MB/s eta 0:00:00


In [2]:
# --- 데이터 로드 및 전처리 ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from tqdm.notebook import tqdm
import gc

In [4]:
# === 1. PyTorch 장치 설정 ===
print("🌟 Step 0: PyTorch 장치 설정")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 사용 가능한 장치: {DEVICE}")

🌟 Step 0: PyTorch 장치 설정
✅ 사용 가능한 장치: cuda


# Amazon

## T5

In [5]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_T5_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 14:30:41,478] A new study created in memory with name: no-name-63f12785-111f-4863-9bc5-1cb4c89701a3


✅ 완료 (학습용: 71941건, 테스트용: 17986건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 14:31:35,473] Trial 0 finished with value: 0.2981652789399747 and parameters: {'n_layers': 2, 'n_units_l0': 186, 'dropout_l0': 0.15892758484028113, 'n_units_l1': 257, 'dropout_l1': 0.47172771757275556, 'learning_rate': 3.647789321560909e-05}. Best is trial 0 with value: 0.2981652789399747.
[I 2025-10-12 14:32:20,304] Trial 1 finished with value: 0.29751379988279913 and parameters: {'n_layers': 1, 'n_units_l0': 114, 'dropout_l0': 0.3671092966933941, 'learning_rate': 9.825332102664499e-05}. Best is trial 0 with value: 0.2981652789399747.
[I 2025-10-12 14:32:59,988] Trial 2 finished with value: 0.3010748943694691 and parameters: {'n_layers': 2, 'n_units_l0': 206, 'dropout_l0': 0.13969848886970945, 'n_units_l1': 372, 'dropout_l1': 0.1794383679356839, 'learning_rate': 0.00013127777816374458}. Best is trial 2 with value: 0.3010748943694691.
[I 2025-10-12 14:33:17,468] Trial 3 finished with value: 0.29871010172570645 and parameters: {'n_layers': 4, 'n_units_l0': 78, 'dropout_l0'


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 3
          n_units_l0: 266
          dropout_l0: 0.43258733866962806
          n_units_l1: 291
          dropout_l1: 0.2865953307130984
          n_units_l2: 56
          dropout_l2: 0.2233978579549068
       learning_rate: 0.000930542552997091

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.3353   0.7806    0.1173

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/176 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_T5_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [6]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_BERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 14:50:42,389] A new study created in memory with name: no-name-6c59bb5f-f03a-495e-9f13-1ef2a577b709


✅ 완료 (학습용: 71941건, 테스트용: 17986건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 14:50:59,631] Trial 0 finished with value: 0.3005325573125909 and parameters: {'n_layers': 4, 'n_units_l0': 418, 'dropout_l0': 0.22598653784351652, 'n_units_l1': 101, 'dropout_l1': 0.13485756597182183, 'n_units_l2': 43, 'dropout_l2': 0.1362852945645745, 'n_units_l3': 71, 'dropout_l3': 0.36775696596434626, 'learning_rate': 0.00015896499047781174}. Best is trial 0 with value: 0.3005325573125909.
[I 2025-10-12 14:51:48,276] Trial 1 finished with value: 0.30147466741629514 and parameters: {'n_layers': 2, 'n_units_l0': 231, 'dropout_l0': 0.17875369314235817, 'n_units_l1': 160, 'dropout_l1': 0.4631193760591561, 'learning_rate': 2.6718819670902943e-05}. Best is trial 1 with value: 0.30147466741629514.
[I 2025-10-12 14:51:56,142] Trial 2 finished with value: 0.29194708797669544 and parameters: {'n_layers': 2, 'n_units_l0': 52, 'dropout_l0': 0.45503153496806636, 'n_units_l1': 151, 'dropout_l1': 0.3543790415166663, 'learning_rate': 0.0034637209524146593}. Best is trial 1 with value


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 2
          n_units_l0: 231
          dropout_l0: 0.17875369314235817
          n_units_l1: 160
          dropout_l1: 0.4631193760591561
       learning_rate: 2.6718819670902943e-05

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch   0.336   0.7833    0.1616

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/176 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_BERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [7]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_SentenceBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 14:53:52,339] A new study created in memory with name: no-name-7019b99f-3683-4051-87ab-7bae22379829


✅ 완료 (학습용: 71941건, 테스트용: 17986건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 14:54:47,119] Trial 0 finished with value: 0.28384230773173325 and parameters: {'n_layers': 4, 'n_units_l0': 40, 'dropout_l0': 0.1998555534075801, 'n_units_l1': 33, 'dropout_l1': 0.44396616215079554, 'n_units_l2': 425, 'dropout_l2': 0.24455731711932327, 'n_units_l3': 66, 'dropout_l3': 0.22478133080450058, 'learning_rate': 4.1909688791150176e-05}. Best is trial 0 with value: 0.28384230773173325.
[I 2025-10-12 14:55:21,332] Trial 1 finished with value: 0.2893542310575601 and parameters: {'n_layers': 4, 'n_units_l0': 159, 'dropout_l0': 0.1581513004836418, 'n_units_l1': 353, 'dropout_l1': 0.23417547648060721, 'n_units_l2': 334, 'dropout_l2': 0.1724096880815517, 'n_units_l3': 228, 'dropout_l3': 0.4809148527093007, 'learning_rate': 3.550282259913115e-05}. Best is trial 1 with value: 0.2893542310575601.
[I 2025-10-12 14:55:29,913] Trial 2 finished with value: 0.2832942709380887 and parameters: {'n_layers': 5, 'n_units_l0': 102, 'dropout_l0': 0.12556208649505712, 'n_units_l1': 44


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 5
          n_units_l0: 112
          dropout_l0: 0.33357835414759074
          n_units_l1: 130
          dropout_l1: 0.2992151001226745
          n_units_l2: 45
          dropout_l2: 0.14063689901594803
          n_units_l3: 203
          dropout_l3: 0.26896689207005764
          n_units_l4: 63
          dropout_l4: 0.4789025864453683
       learning_rate: 0.00011663153685783594

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.2848   0.7509    0.2001

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/176 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_SentenceBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [8]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_RoBERTa_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 14:58:16,843] A new study created in memory with name: no-name-12b08e26-38bb-439a-86e4-02e163e79668


✅ 완료 (학습용: 71941건, 테스트용: 17986건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 14:58:40,411] Trial 0 finished with value: 0.29353196431320183 and parameters: {'n_layers': 2, 'n_units_l0': 90, 'dropout_l0': 0.25233379653788296, 'n_units_l1': 99, 'dropout_l1': 0.4513697007175935, 'learning_rate': 8.591778394097837e-05}. Best is trial 0 with value: 0.29353196431320183.
[I 2025-10-12 14:59:19,395] Trial 1 finished with value: 0.29109673192298186 and parameters: {'n_layers': 1, 'n_units_l0': 50, 'dropout_l0': 0.39323749953951725, 'learning_rate': 5.4289641319051e-05}. Best is trial 0 with value: 0.29353196431320183.
[I 2025-10-12 14:59:34,539] Trial 2 finished with value: 0.2983929872204086 and parameters: {'n_layers': 5, 'n_units_l0': 151, 'dropout_l0': 0.33541419535763534, 'n_units_l1': 368, 'dropout_l1': 0.3463654117939637, 'n_units_l2': 449, 'dropout_l2': 0.42893551897915216, 'n_units_l3': 263, 'dropout_l3': 0.34434303569521374, 'n_units_l4': 377, 'dropout_l4': 0.40517616228682607, 'learning_rate': 0.0008194663683445672}. Best is trial 2 with value: 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 5
          n_units_l0: 116
          dropout_l0: 0.2885002245900376
          n_units_l1: 352
          dropout_l1: 0.36109284010178866
          n_units_l2: 336
          dropout_l2: 0.3949540459693273
          n_units_l3: 303
          dropout_l3: 0.2827995283642472
          n_units_l4: 156
          dropout_l4: 0.4961550340547851
       learning_rate: 0.0007766702220744428

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.3212   0.7558    0.1194

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/176 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_RoBERTa_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [9]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/amazon/amazon.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/amazon_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_DistilBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:05:18,415] A new study created in memory with name: no-name-3f50cccf-8847-4b46-bba4-f86381f0f51c


✅ 완료 (학습용: 71941건, 테스트용: 17986건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:05:31,505] Trial 0 finished with value: 0.2985317619185047 and parameters: {'n_layers': 5, 'n_units_l0': 221, 'dropout_l0': 0.25802259904852454, 'n_units_l1': 199, 'dropout_l1': 0.42851237140329723, 'n_units_l2': 254, 'dropout_l2': 0.3017183766637207, 'n_units_l3': 64, 'dropout_l3': 0.32758550554287097, 'n_units_l4': 243, 'dropout_l4': 0.1776006257134057, 'learning_rate': 0.0008298681273709458}. Best is trial 0 with value: 0.2985317619185047.
[I 2025-10-12 15:06:02,575] Trial 1 finished with value: 0.29909116710192185 and parameters: {'n_layers': 4, 'n_units_l0': 37, 'dropout_l0': 0.19544813302795672, 'n_units_l1': 221, 'dropout_l1': 0.43281807462347877, 'n_units_l2': 289, 'dropout_l2': 0.24524047806936944, 'n_units_l3': 252, 'dropout_l3': 0.22434050578492182, 'learning_rate': 6.724060977499696e-05}. Best is trial 1 with value: 0.29909116710192185.
[I 2025-10-12 15:06:18,402] Trial 2 finished with value: 0.29776294929409663 and parameters: {'n_layers': 2, 'n_units_l0':


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 3
          n_units_l0: 182
          dropout_l0: 0.21928302736213126
          n_units_l1: 272
          dropout_l1: 0.31986279742139023
          n_units_l2: 297
          dropout_l2: 0.31941296712554257
       learning_rate: 0.0009392363424652996

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.2476   0.7408    0.1151

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/176 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/amazon/MLP/MLP_amazon_with_DistilBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Coursera

## T5

In [10]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_T5_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:08:44,061] A new study created in memory with name: no-name-3939b791-0d4b-4240-b66e-6e01111cad50


✅ 완료 (학습용: 97108건, 테스트용: 24278건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:09:09,054] Trial 0 finished with value: 0.2223024103973823 and parameters: {'n_layers': 3, 'n_units_l0': 110, 'dropout_l0': 0.2652667823566912, 'n_units_l1': 417, 'dropout_l1': 0.21959280501933698, 'n_units_l2': 378, 'dropout_l2': 0.3935111133916508, 'learning_rate': 0.0017610272144902785}. Best is trial 0 with value: 0.2223024103973823.
[I 2025-10-12 15:09:28,765] Trial 1 finished with value: 0.2165700348558064 and parameters: {'n_layers': 3, 'n_units_l0': 35, 'dropout_l0': 0.37246362152018997, 'n_units_l1': 215, 'dropout_l1': 0.4866734467770705, 'n_units_l2': 200, 'dropout_l2': 0.4271909418748937, 'learning_rate': 0.0029372749657421947}. Best is trial 0 with value: 0.2223024103973823.
[I 2025-10-12 15:10:30,389] Trial 2 finished with value: 0.20929307689319668 and parameters: {'n_layers': 1, 'n_units_l0': 41, 'dropout_l0': 0.30509547556566474, 'learning_rate': 8.607634092007127e-05}. Best is trial 0 with value: 0.2223024103973823.
[I 2025-10-12 15:11:43,657] Trial 3 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 3
          n_units_l0: 110
          dropout_l0: 0.2652667823566912
          n_units_l1: 417
          dropout_l1: 0.21959280501933698
          n_units_l2: 378
          dropout_l2: 0.3935111133916508
       learning_rate: 0.0017610272144902785

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch   0.211   0.7748    0.1305

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/238 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_T5_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [11]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_BERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:15:08,540] A new study created in memory with name: no-name-cf4a3565-96f3-4d0e-a98d-91b3a3a59cde


✅ 완료 (학습용: 97108건, 테스트용: 24278건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:15:45,376] Trial 0 finished with value: 0.20532492844763076 and parameters: {'n_layers': 4, 'n_units_l0': 131, 'dropout_l0': 0.12482202061334227, 'n_units_l1': 75, 'dropout_l1': 0.12868744111960156, 'n_units_l2': 217, 'dropout_l2': 0.48004912675220746, 'n_units_l3': 49, 'dropout_l3': 0.35750445309977996, 'learning_rate': 4.269764556598191e-05}. Best is trial 0 with value: 0.20532492844763076.
[I 2025-10-12 15:16:17,983] Trial 1 finished with value: 0.20757549195513514 and parameters: {'n_layers': 5, 'n_units_l0': 494, 'dropout_l0': 0.16666537867677733, 'n_units_l1': 55, 'dropout_l1': 0.3681928583838223, 'n_units_l2': 278, 'dropout_l2': 0.4262919237467758, 'n_units_l3': 162, 'dropout_l3': 0.324821149297923, 'n_units_l4': 205, 'dropout_l4': 0.4925078822514204, 'learning_rate': 4.726569318383233e-05}. Best is trial 1 with value: 0.20757549195513514.
[I 2025-10-12 15:16:29,943] Trial 2 finished with value: 0.2049826451557517 and parameters: {'n_layers': 2, 'n_units_l0': 58


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 1
          n_units_l0: 140
          dropout_l0: 0.3386993188683119
       learning_rate: 8.622367446973934e-05

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch   0.223   0.7879    0.0887

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/238 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_BERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [12]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_SentenceBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:20:46,065] A new study created in memory with name: no-name-e41359b1-5274-47af-a9ed-d204c5c4f1af


✅ 완료 (학습용: 97108건, 테스트용: 24278건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:22:01,006] Trial 0 finished with value: 0.18267204495701522 and parameters: {'n_layers': 4, 'n_units_l0': 512, 'dropout_l0': 0.1377973729229478, 'n_units_l1': 114, 'dropout_l1': 0.1529118149701358, 'n_units_l2': 50, 'dropout_l2': 0.31009664530941905, 'n_units_l3': 134, 'dropout_l3': 0.12083983890278925, 'learning_rate': 1.1699308115936522e-05}. Best is trial 0 with value: 0.18267204495701522.
[I 2025-10-12 15:22:13,261] Trial 1 finished with value: 0.19170557484759362 and parameters: {'n_layers': 2, 'n_units_l0': 39, 'dropout_l0': 0.21543783989588491, 'n_units_l1': 56, 'dropout_l1': 0.43765742076895686, 'learning_rate': 0.0031399831590192503}. Best is trial 1 with value: 0.19170557484759362.
[I 2025-10-12 15:22:28,652] Trial 2 finished with value: 0.18973976049919655 and parameters: {'n_layers': 1, 'n_units_l0': 132, 'dropout_l0': 0.4874060650939248, 'learning_rate': 0.0036575664418538505}. Best is trial 1 with value: 0.19170557484759362.
[I 2025-10-12 15:23:20,468] Tr


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 2
          n_units_l0: 39
          dropout_l0: 0.21543783989588491
          n_units_l1: 56
          dropout_l1: 0.43765742076895686
       learning_rate: 0.0031399831590192503

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1561   0.6913    0.1385

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/238 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_SentenceBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [13]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_RoBERTa_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:27:07,173] A new study created in memory with name: no-name-40dd7485-5e04-4c4e-b919-66877a1b71e7


✅ 완료 (학습용: 97108건, 테스트용: 24278건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:27:22,928] Trial 0 finished with value: 0.20152063545381013 and parameters: {'n_layers': 3, 'n_units_l0': 194, 'dropout_l0': 0.4807034430037874, 'n_units_l1': 140, 'dropout_l1': 0.34867125300516777, 'n_units_l2': 42, 'dropout_l2': 0.2682347826873447, 'learning_rate': 0.0010824089250375361}. Best is trial 0 with value: 0.20152063545381013.
[I 2025-10-12 15:28:09,960] Trial 1 finished with value: 0.21239474606165548 and parameters: {'n_layers': 5, 'n_units_l0': 149, 'dropout_l0': 0.4594702681749995, 'n_units_l1': 124, 'dropout_l1': 0.2543661780328996, 'n_units_l2': 59, 'dropout_l2': 0.18705462757355937, 'n_units_l3': 63, 'dropout_l3': 0.38430870990778554, 'n_units_l4': 235, 'dropout_l4': 0.4251338061038049, 'learning_rate': 0.000752448148146284}. Best is trial 1 with value: 0.21239474606165548.
[I 2025-10-12 15:29:12,389] Trial 2 finished with value: 0.21053398935116582 and parameters: {'n_layers': 1, 'n_units_l0': 347, 'dropout_l0': 0.48227153062790584, 'learning_rate':


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 4
          n_units_l0: 133
          dropout_l0: 0.4717165261437468
          n_units_l1: 79
          dropout_l1: 0.10205448275752255
          n_units_l2: 97
          dropout_l2: 0.40188795124183274
          n_units_l3: 153
          dropout_l3: 0.18391629125407027
       learning_rate: 0.0001856344822916662

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch   0.213   0.7646    0.1071

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/238 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_RoBERTa_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [14]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/coursera/coursera.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/coursera_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_DistilBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:34:20,544] A new study created in memory with name: no-name-07e7ebb5-a150-437d-8b29-e4c6d7b41149


✅ 완료 (학습용: 97108건, 테스트용: 24278건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:34:37,852] Trial 0 finished with value: 0.19956035701172847 and parameters: {'n_layers': 5, 'n_units_l0': 73, 'dropout_l0': 0.18504176280155585, 'n_units_l1': 279, 'dropout_l1': 0.1944009346027989, 'n_units_l2': 281, 'dropout_l2': 0.15203786829464586, 'n_units_l3': 32, 'dropout_l3': 0.14476559790303756, 'n_units_l4': 52, 'dropout_l4': 0.4673966585341116, 'learning_rate': 0.002391994372585765}. Best is trial 0 with value: 0.19956035701172847.
[I 2025-10-12 15:35:03,045] Trial 1 finished with value: 0.21032719026145774 and parameters: {'n_layers': 1, 'n_units_l0': 344, 'dropout_l0': 0.3430771021245596, 'learning_rate': 0.0008476866845825352}. Best is trial 1 with value: 0.21032719026145774.
[I 2025-10-12 15:36:05,875] Trial 2 finished with value: 0.20454568568899742 and parameters: {'n_layers': 1, 'n_units_l0': 38, 'dropout_l0': 0.2691742990119146, 'learning_rate': 0.00011572694016555544}. Best is trial 1 with value: 0.21032719026145774.
[I 2025-10-12 15:36:37,301] Trial


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 1
          n_units_l0: 344
          dropout_l0: 0.3430771021245596
       learning_rate: 0.0008476866845825352

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.2064    0.772    0.1095

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/238 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/coursera/MLP/MLP_coursera_with_DistilBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Audible

## T5

In [15]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_T5_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:39:14,705] A new study created in memory with name: no-name-add98394-d969-49ed-9ea8-b6e5c736077a


✅ 완료 (학습용: 74391건, 테스트용: 18598건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:39:38,687] Trial 0 finished with value: 0.15832367005918896 and parameters: {'n_layers': 4, 'n_units_l0': 334, 'dropout_l0': 0.4374513967225664, 'n_units_l1': 287, 'dropout_l1': 0.2441970909381669, 'n_units_l2': 176, 'dropout_l2': 0.2085248071529681, 'n_units_l3': 119, 'dropout_l3': 0.40055546378176543, 'learning_rate': 0.00017562084192594755}. Best is trial 0 with value: 0.15832367005918896.
[I 2025-10-12 15:39:51,159] Trial 1 finished with value: 0.15755277964706466 and parameters: {'n_layers': 1, 'n_units_l0': 119, 'dropout_l0': 0.24933531147195281, 'learning_rate': 0.007756214405997634}. Best is trial 0 with value: 0.15832367005918896.
[I 2025-10-12 15:40:39,421] Trial 2 finished with value: 0.16160437789424167 and parameters: {'n_layers': 1, 'n_units_l0': 395, 'dropout_l0': 0.10573661005825641, 'learning_rate': 9.033407885502957e-05}. Best is trial 2 with value: 0.16160437789424167.
[I 2025-10-12 15:41:25,141] Trial 3 finished with value: 0.1569357274098782 and pa


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 2
          n_units_l0: 495
          dropout_l0: 0.33441689932313784
          n_units_l1: 87
          dropout_l1: 0.27860212361297026
       learning_rate: 0.0021077341581684705

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1695   0.6799    0.0073

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/182 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_T5_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [16]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_BERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:47:40,435] A new study created in memory with name: no-name-eb23bf5b-7e83-4892-8938-b6ab68082287


✅ 완료 (학습용: 74391건, 테스트용: 18598건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:48:24,992] Trial 0 finished with value: 0.15192347228003172 and parameters: {'n_layers': 1, 'n_units_l0': 113, 'dropout_l0': 0.4266391290773144, 'learning_rate': 6.737352817892477e-05}. Best is trial 0 with value: 0.15192347228003172.
[I 2025-10-12 15:49:13,078] Trial 1 finished with value: 0.1465675852809513 and parameters: {'n_layers': 1, 'n_units_l0': 85, 'dropout_l0': 0.16891559285256597, 'learning_rate': 2.7830442058512113e-05}. Best is trial 0 with value: 0.15192347228003172.
[I 2025-10-12 15:50:04,454] Trial 2 finished with value: 0.1541960963431384 and parameters: {'n_layers': 2, 'n_units_l0': 35, 'dropout_l0': 0.3104531674853326, 'n_units_l1': 131, 'dropout_l1': 0.19173763166862623, 'learning_rate': 7.205981056702433e-05}. Best is trial 2 with value: 0.1541960963431384.
[I 2025-10-12 15:50:27,270] Trial 3 finished with value: 0.15127802936876047 and parameters: {'n_layers': 3, 'n_units_l0': 219, 'dropout_l0': 0.46224212479192617, 'n_units_l1': 298, 'dropout_l1


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 2
          n_units_l0: 35
          dropout_l0: 0.3104531674853326
          n_units_l1: 131
          dropout_l1: 0.19173763166862623
       learning_rate: 7.205981056702433e-05

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1598   0.6711       0.0

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/182 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_BERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [17]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_SentenceBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:52:52,213] A new study created in memory with name: no-name-177ff7b4-ba18-4b9f-94dd-3eb6ecdac401


✅ 완료 (학습용: 74391건, 테스트용: 18598건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:53:00,136] Trial 0 finished with value: 0.14274299459979978 and parameters: {'n_layers': 1, 'n_units_l0': 487, 'dropout_l0': 0.45139303578864476, 'learning_rate': 0.007236977124199203}. Best is trial 0 with value: 0.14274299459979978.
[I 2025-10-12 15:54:00,763] Trial 1 finished with value: 0.13933499637589408 and parameters: {'n_layers': 5, 'n_units_l0': 44, 'dropout_l0': 0.18041571475643053, 'n_units_l1': 215, 'dropout_l1': 0.19838884725967107, 'n_units_l2': 46, 'dropout_l2': 0.41204714975308, 'n_units_l3': 62, 'dropout_l3': 0.22267341299817814, 'n_units_l4': 61, 'dropout_l4': 0.13538531395420722, 'learning_rate': 2.1675754163090682e-05}. Best is trial 0 with value: 0.14274299459979978.
[I 2025-10-12 15:54:58,619] Trial 2 finished with value: 0.1373829923412832 and parameters: {'n_layers': 4, 'n_units_l0': 59, 'dropout_l0': 0.3403626530764686, 'n_units_l1': 510, 'dropout_l1': 0.22659566107586193, 'n_units_l2': 38, 'dropout_l2': 0.47471210345369197, 'n_units_l3': 165,


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 1
          n_units_l0: 510
          dropout_l0: 0.48690023179074754
       learning_rate: 0.0013577352771724437

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1335   0.6256    0.0513

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/182 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_SentenceBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [18]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_RoBERTa_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 15:58:16,160] A new study created in memory with name: no-name-eb07c62e-56c0-4652-9561-eded89fb6207


✅ 완료 (학습용: 74391건, 테스트용: 18598건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 15:58:24,705] Trial 0 finished with value: 0.13174112968441126 and parameters: {'n_layers': 5, 'n_units_l0': 58, 'dropout_l0': 0.1872427037226173, 'n_units_l1': 41, 'dropout_l1': 0.16937730376870713, 'n_units_l2': 202, 'dropout_l2': 0.188976356249764, 'n_units_l3': 53, 'dropout_l3': 0.42253734889555805, 'n_units_l4': 107, 'dropout_l4': 0.45965439544069386, 'learning_rate': 0.007404959507944304}. Best is trial 0 with value: 0.13174112968441126.
[I 2025-10-12 15:58:57,521] Trial 1 finished with value: 0.16151646996896424 and parameters: {'n_layers': 3, 'n_units_l0': 45, 'dropout_l0': 0.43854356508708436, 'n_units_l1': 139, 'dropout_l1': 0.4815581827609485, 'n_units_l2': 256, 'dropout_l2': 0.47767365546224894, 'learning_rate': 0.0002771045602238846}. Best is trial 1 with value: 0.16151646996896424.
[I 2025-10-12 15:59:45,848] Trial 2 finished with value: 0.15098698276991518 and parameters: {'n_layers': 1, 'n_units_l0': 318, 'dropout_l0': 0.1870650144335862, 'learning_rate': 


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 3
          n_units_l0: 45
          dropout_l0: 0.43854356508708436
          n_units_l1: 139
          dropout_l1: 0.4815581827609485
          n_units_l2: 256
          dropout_l2: 0.47767365546224894
       learning_rate: 0.0002771045602238846

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1642   0.6686       0.0

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/182 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_RoBERTa_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [19]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/audible/audible.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/audible_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_DistilBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:03:04,333] A new study created in memory with name: no-name-d1ca03ac-c021-47e4-b83a-cca013693f19


✅ 완료 (학습용: 74391건, 테스트용: 18598건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:03:11,788] Trial 0 finished with value: 0.13825621897110735 and parameters: {'n_layers': 5, 'n_units_l0': 100, 'dropout_l0': 0.31830034455787315, 'n_units_l1': 331, 'dropout_l1': 0.3558231209432291, 'n_units_l2': 162, 'dropout_l2': 0.16116065334673768, 'n_units_l3': 248, 'dropout_l3': 0.26019555960527124, 'n_units_l4': 274, 'dropout_l4': 0.4775818024222245, 'learning_rate': 0.003335294747810453}. Best is trial 0 with value: 0.13825621897110735.
[I 2025-10-12 16:03:35,972] Trial 1 finished with value: 0.15401570025082523 and parameters: {'n_layers': 5, 'n_units_l0': 319, 'dropout_l0': 0.3010126367862981, 'n_units_l1': 40, 'dropout_l1': 0.33554788935568425, 'n_units_l2': 203, 'dropout_l2': 0.46984624774219397, 'n_units_l3': 77, 'dropout_l3': 0.3838172560202051, 'n_units_l4': 45, 'dropout_l4': 0.43901527711761834, 'learning_rate': 0.0009127294789640938}. Best is trial 1 with value: 0.15401570025082523.
[I 2025-10-12 16:03:55,654] Trial 2 finished with value: 0.15460965771


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 4
          n_units_l0: 63
          dropout_l0: 0.10338976975277406
          n_units_l1: 179
          dropout_l1: 0.14004911185529656
          n_units_l2: 125
          dropout_l2: 0.34161489081016616
          n_units_l3: 200
          dropout_l3: 0.23758740993938174
       learning_rate: 0.0004524147038511728

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1204   0.6192    0.1106

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/182 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/audible/MLP/MLP_audible_with_DistilBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


# Hotel

## T5

In [20]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_T5.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_T5_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:06:01,286] A new study created in memory with name: no-name-3b496b69-0dc1-434f-a1e2-c816c87064a4


✅ 완료 (학습용: 71604건, 테스트용: 17901건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:06:37,139] Trial 0 finished with value: 0.15161856912473623 and parameters: {'n_layers': 2, 'n_units_l0': 204, 'dropout_l0': 0.38539775517970076, 'n_units_l1': 37, 'dropout_l1': 0.21168123999466634, 'learning_rate': 0.0002877963856049959}. Best is trial 0 with value: 0.15161856912473623.
[I 2025-10-12 16:06:57,186] Trial 1 finished with value: 0.1528711980009655 and parameters: {'n_layers': 3, 'n_units_l0': 122, 'dropout_l0': 0.27529916434164037, 'n_units_l1': 230, 'dropout_l1': 0.4897739270964866, 'n_units_l2': 45, 'dropout_l2': 0.3621199109140255, 'learning_rate': 0.003349912006873451}. Best is trial 1 with value: 0.1528711980009655.
[I 2025-10-12 16:07:43,557] Trial 2 finished with value: 0.15006523292398286 and parameters: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.3329626027025907, 'learning_rate': 2.8862559756640022e-05}. Best is trial 1 with value: 0.1528711980009655.
[I 2025-10-12 16:08:02,234] Trial 3 finished with value: 0.15148220973489562 and parame


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 4
          n_units_l0: 494
          dropout_l0: 0.24049679668559834
          n_units_l1: 97
          dropout_l1: 0.1564248789370703
          n_units_l2: 104
          dropout_l2: 0.25542965885665486
          n_units_l3: 301
          dropout_l3: 0.3685415777085128
       learning_rate: 0.0008039619017433245

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1427   0.6092    0.0036

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/175 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_T5_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## BERT

In [21]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_BERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_BERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:12:49,898] A new study created in memory with name: no-name-657a184c-851a-46cc-b0e2-6db5f873e222


✅ 완료 (학습용: 71604건, 테스트용: 17901건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:12:58,447] Trial 0 finished with value: 0.15443806758029965 and parameters: {'n_layers': 3, 'n_units_l0': 132, 'dropout_l0': 0.3829208090034567, 'n_units_l1': 166, 'dropout_l1': 0.4060705367536662, 'n_units_l2': 113, 'dropout_l2': 0.18885584021139606, 'learning_rate': 0.0007189667171637848}. Best is trial 0 with value: 0.15443806758029965.
[I 2025-10-12 16:13:04,546] Trial 1 finished with value: 0.15431950651794485 and parameters: {'n_layers': 2, 'n_units_l0': 90, 'dropout_l0': 0.43045288632302003, 'n_units_l1': 88, 'dropout_l1': 0.3850870912237738, 'learning_rate': 0.004143467874781397}. Best is trial 0 with value: 0.15443806758029965.
[I 2025-10-12 16:13:12,079] Trial 2 finished with value: 0.15485281270657641 and parameters: {'n_layers': 1, 'n_units_l0': 105, 'dropout_l0': 0.1413700523847067, 'learning_rate': 0.0004884039439928777}. Best is trial 2 with value: 0.15485281270657641.
[I 2025-10-12 16:13:24,822] Trial 3 finished with value: 0.15644305603190206 and param


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 3
          n_units_l0: 442
          dropout_l0: 0.16857907069749684
          n_units_l1: 176
          dropout_l1: 0.1120750139980193
          n_units_l2: 165
          dropout_l2: 0.20016020252389768
       learning_rate: 4.732208733802798e-05

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1289   0.5936    0.0358

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/175 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_BERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## SentenceBERT

In [22]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_SentenceBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_SentenceBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:14:58,801] A new study created in memory with name: no-name-c0c1fe7b-ed57-4609-845f-42b075f6e465


✅ 완료 (학습용: 71604건, 테스트용: 17901건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:15:45,827] Trial 0 finished with value: 0.1394514622129316 and parameters: {'n_layers': 1, 'n_units_l0': 52, 'dropout_l0': 0.24493390119485922, 'learning_rate': 5.818679387726616e-05}. Best is trial 0 with value: 0.1394514622129316.
[I 2025-10-12 16:15:54,216] Trial 1 finished with value: 0.14646327296663497 and parameters: {'n_layers': 5, 'n_units_l0': 347, 'dropout_l0': 0.20801956196523008, 'n_units_l1': 36, 'dropout_l1': 0.24089193072600193, 'n_units_l2': 34, 'dropout_l2': 0.11682141150713142, 'n_units_l3': 95, 'dropout_l3': 0.41770891758899786, 'n_units_l4': 214, 'dropout_l4': 0.186530464230526, 'learning_rate': 0.006815571577882868}. Best is trial 1 with value: 0.14646327296663497.
[I 2025-10-12 16:16:35,760] Trial 2 finished with value: 0.14945227143781514 and parameters: {'n_layers': 3, 'n_units_l0': 36, 'dropout_l0': 0.3067426899340481, 'n_units_l1': 53, 'dropout_l1': 0.49930225763893876, 'n_units_l2': 54, 'dropout_l2': 0.4954321943107519, 'learning_rate': 9.25


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 4
          n_units_l0: 115
          dropout_l0: 0.10253259669850084
          n_units_l1: 73
          dropout_l1: 0.12053414413052671
          n_units_l2: 368
          dropout_l2: 0.36311543015501163
          n_units_l3: 35
          dropout_l3: 0.10746081457551612
       learning_rate: 0.0005774385668473865

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1022   0.5317    0.0544

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/175 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_SentenceBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## RoBERTa

In [23]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_RoBERTa.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_RoBERTa_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:20:24,422] A new study created in memory with name: no-name-148c1b7b-ae25-4308-abec-b1833c6a043c


✅ 완료 (학습용: 71604건, 테스트용: 17901건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:20:59,857] Trial 0 finished with value: 0.15153922519140822 and parameters: {'n_layers': 5, 'n_units_l0': 40, 'dropout_l0': 0.4141097242177517, 'n_units_l1': 315, 'dropout_l1': 0.1700179250458588, 'n_units_l2': 181, 'dropout_l2': 0.19070559408417234, 'n_units_l3': 42, 'dropout_l3': 0.22753625197916488, 'n_units_l4': 247, 'dropout_l4': 0.10506216334006711, 'learning_rate': 5.369980220053131e-05}. Best is trial 0 with value: 0.15153922519140822.
[I 2025-10-12 16:21:12,705] Trial 1 finished with value: 0.15160225054489201 and parameters: {'n_layers': 3, 'n_units_l0': 277, 'dropout_l0': 0.3507966483686322, 'n_units_l1': 49, 'dropout_l1': 0.49960437532807567, 'n_units_l2': 49, 'dropout_l2': 0.15885209008467438, 'learning_rate': 0.00571465090801897}. Best is trial 1 with value: 0.15160225054489201.
[I 2025-10-12 16:21:34,997] Trial 2 finished with value: 0.1510743327761971 and parameters: {'n_layers': 4, 'n_units_l0': 498, 'dropout_l0': 0.16365101283635924, 'n_units_l1': 203


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 2
          n_units_l0: 36
          dropout_l0: 0.13268992421002965
          n_units_l1: 133
          dropout_l1: 0.2358006169880642
       learning_rate: 0.00040597131085601674

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1463   0.6094    0.0107

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/175 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_RoBERTa_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!


## DistilBERT

In [24]:
# === 2. 환경설정 클래스 ===
class Config:
    """실행에 필요한 모든 설정값을 중앙에서 관리합니다."""
    # 🌟 1. 파일 경로 설정
    CSV_FILE_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/data/hotel/hotel.csv"
    EMBEDDING_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/embedding/hotel_DistilBERT.npy"

    # 🌟 2. 최종 결과 CSV 파일 저장 경로 설정
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_DistilBERT_predictions.csv"

    # --- 데이터 정보 ---
    TARGET_COLUMN = 'binary_helpfulness'

    # --- 데이터 분할 ---
    TEST_SPLIT_RATIO = 0.2
    RANDOM_STATE = 42

    # --- PyTorch 모델 및 학습 설정 ---
    EPOCHS = 50
    BATCH_SIZE = 256
    VALIDATION_EARLY_STOPPING_PATIENCE = 5 # 개별 Trial 내 검증 성능 기반 조기 종료

    # --- Optuna 튜닝 설정 ---
    N_TRIALS = 50
    TUNING_METRIC = 'pr_auc'
    STUDY_EARLY_STOPPING_ROUNDS = 10 # 전체 Study 조기 종료

# === 3. MLP 모델 클래스 (PyTorch) ===
class MLP(nn.Module):
    def __init__(self, input_dim, trial):
        super(MLP, self).__init__()
        layers = []

        # ✅ 은닉층 탐색 범위를 1~5개로 확장
        n_layers = trial.suggest_int('n_layers', 1, 5)

        in_features = input_dim
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            p = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
            layers.append(nn.Dropout(p))
            in_features = out_features

        layers.append(nn.Linear(in_features, 1))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.layers(x).squeeze(-1))

# === 4. 학습 및 평가 함수 ===
def train_model(model, loader, optimizer, criterion):
    model.train()
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# === 5. Optuna Objective 함수 (PyTorch용) ===
def objective(trial, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=Config.RANDOM_STATE, stratify=y)

    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = MLP(input_dim, trial).to(DEVICE)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    best_score = -1
    patience_counter = 0

    for epoch in range(Config.EPOCHS):
        train_model(model, train_loader, optimizer, criterion)
        y_pred_proba, y_true = evaluate_model(model, val_loader)
        score = average_precision_score(y_true, y_pred_proba)

        trial.report(score, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if score > best_score:
            best_score = score
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= Config.VALIDATION_EARLY_STOPPING_PATIENCE:
            break

    return best_score

# === 6. Optuna 조기 종료 콜백 ===
class EarlyStoppingCallback:
    def __init__(self, early_stopping_rounds: int):
        self._early_stopping_rounds = early_stopping_rounds
        self._best_value = -float("inf")
        self._counter = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.Trial):
        current_best_value = study.best_value
        if current_best_value is not None and current_best_value > self._best_value:
            self._best_value = current_best_value
            self._counter = 0
        else:
            self._counter += 1

        if self._counter >= self._early_stopping_rounds:
            print(f"\n[Optuna 조기 종료] {self._early_stopping_rounds}번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.")
            study.stop()


# === 7. 메인 실행 블록 ===
if __name__ == '__main__':
    config = Config()

    # --- Step 1: 데이터 로드 및 분할 ---
    print("\n" + "="*50)
    print("📊 Step 1: 데이터 로드 및 분할")
    try:
        df = pd.read_csv(config.CSV_FILE_PATH)
        labels = df[config.TARGET_COLUMN].values
        embeddings = np.load(config.EMBEDDING_PATH)
    except Exception as e:
        print(f"🔥 파일 로드 실패: {e}"); exit()

    indices = np.arange(len(df))
    train_indices, test_indices = train_test_split(
        indices, test_size=config.TEST_SPLIT_RATIO, random_state=config.RANDOM_STATE, stratify=labels)
    X_train, X_test = embeddings[train_indices], embeddings[test_indices]
    y_train, y_test = labels[train_indices], labels[test_indices]
    print(f"✅ 완료 (학습용: {len(y_train)}건, 테스트용: {len(y_test)}건)")

    # --- Step 2: Optuna 튜닝 수행 ---
    print("\n" + "="*50)
    print(f"🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)")
    print(f"(최대 {config.N_TRIALS}번 시도, {config.STUDY_EARLY_STOPPING_ROUNDS}번 개선 없으면 스터디 조기 종료)")
    print("="*50)

    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    pbar = tqdm(total=config.N_TRIALS, desc="Optuna 튜닝 진행률")

    try:
        study.optimize(lambda trial: objective(trial, X_train, y_train),
                       n_trials=config.N_TRIALS,
                       callbacks=[lambda s, t: pbar.update(1), EarlyStoppingCallback(config.STUDY_EARLY_STOPPING_ROUNDS)])
    except optuna.exceptions.OptunaError:
        pass # 조기 종료 시 예외 처리
    pbar.close()

    # --- Step 3: 최적 모델 학습 및 평가 ---
    print(f"\n✅ 튜닝 완료!")
    print("\n" + "="*50)
    print("🔬 최적 하이퍼파라미터")
    print("="*50)
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"{key:>20s}: {value}")
    print("="*50)

    print(f"\n🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...")
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE)

    # Optuna study 객체를 모의 trial로 사용하여 최종 모델 생성
    final_model = MLP(X_train.shape[1], study.best_trial).to(DEVICE)
    optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    criterion = nn.BCELoss()

    # 최종 모델은 전체 학습 데이터로 학습
    for epoch in tqdm(range(config.EPOCHS), desc="최종 모델 학습"):
        train_model(final_model, train_loader, optimizer, criterion)

    y_pred_proba_tuned, y_true_test = evaluate_model(final_model, test_loader)
    y_pred_class_tuned = (y_pred_proba_tuned > 0.5).astype(int)

    results = {
        "PR AUC": average_precision_score(y_true_test, y_pred_proba_tuned),
        "ROC AUC": roc_auc_score(y_true_test, y_pred_proba_tuned),
        "F1-Score": f1_score(y_true_test, y_pred_class_tuned),
    }
    print("✅ 튜닝된 PyTorch 모델 평가 완료.")
    print(pd.DataFrame(results, index=['Optuna_Tuned_PyTorch']).round(4))

    # --- Step 4: 결과 저장 ---
    print("\n" + "="*60)
    print("💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장")
    print("="*60)

    all_dataset = TensorDataset(torch.FloatTensor(embeddings))
    all_loader = DataLoader(all_dataset, batch_size=config.BATCH_SIZE * 2) # 예측 시에는 더 큰 배치 사용 가능

    final_model.eval()
    all_predictions = []
    with torch.no_grad():
        for (inputs,) in tqdm(all_loader, desc="전체 데이터 예측"):
            inputs = inputs.to(DEVICE)
            outputs = final_model(inputs)
            all_predictions.extend(outputs.cpu().numpy())

    df['s2_pred_proba'] = all_predictions
    df['s2_pred_class'] = (df['s2_pred_proba'] > 0.5).astype(int)

    df.to_csv(config.OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ 모든 데이터의 예측 결과가 '{config.OUTPUT_CSV_PATH}' 파일에 성공적으로 저장되었습니다.")

    # --- Step 5: 메모리 정리 ---
    print("\n" + "="*60)
    print("🧹 Step 5: 메모리 정리")
    print("="*60)
    del df, labels, embeddings, X_train, X_test, y_train, y_test, final_model, study
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("✅ 메모리 정리가 완료되었습니다.")

    print("\n🎉 모든 과정이 완료되었습니다!")


📊 Step 1: 데이터 로드 및 분할


[I 2025-10-12 16:23:51,532] A new study created in memory with name: no-name-8d0dc9e5-0b38-4f99-818a-a9656321d4b4


✅ 완료 (학습용: 71604건, 테스트용: 17901건)

🔬 Step 2: Optuna 하이퍼파라미터 튜닝 시작 (PyTorch)
(최대 50번 시도, 10번 개선 없으면 스터디 조기 종료)


Optuna 튜닝 진행률:   0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-12 16:23:59,809] Trial 0 finished with value: 0.15560327030211793 and parameters: {'n_layers': 2, 'n_units_l0': 177, 'dropout_l0': 0.39519157513504966, 'n_units_l1': 148, 'dropout_l1': 0.2390976361447153, 'learning_rate': 0.006913910242036695}. Best is trial 0 with value: 0.15560327030211793.
[I 2025-10-12 16:24:53,357] Trial 1 finished with value: 0.15618580424512035 and parameters: {'n_layers': 3, 'n_units_l0': 45, 'dropout_l0': 0.16396598870782259, 'n_units_l1': 191, 'dropout_l1': 0.28651652033441466, 'n_units_l2': 196, 'dropout_l2': 0.2716078844980865, 'learning_rate': 1.0938916007472452e-05}. Best is trial 1 with value: 0.15618580424512035.
[I 2025-10-12 16:25:03,898] Trial 2 finished with value: 0.15753210413792404 and parameters: {'n_layers': 1, 'n_units_l0': 212, 'dropout_l0': 0.11523364915600696, 'learning_rate': 0.009251667139183774}. Best is trial 2 with value: 0.15753210413792404.
[I 2025-10-12 16:25:17,499] Trial 3 finished with value: 0.1579904473750274 and par


[Optuna 조기 종료] 10번의 trial 동안 최고 점수가 갱신되지 않아 튜닝을 중단합니다.

✅ 튜닝 완료!

🔬 최적 하이퍼파라미터
            n_layers: 4
          n_units_l0: 302
          dropout_l0: 0.4021283542423828
          n_units_l1: 154
          dropout_l1: 0.1437402693720934
          n_units_l2: 240
          dropout_l2: 0.45059989981884596
          n_units_l3: 36
          dropout_l3: 0.14739806050946044
       learning_rate: 0.00016387652919042103

🔬 Step 3: 튜닝된 최종 PyTorch 모델 학습 및 평가...


최종 모델 학습:   0%|          | 0/50 [00:00<?, ?it/s]

✅ 튜닝된 PyTorch 모델 평가 완료.
                      PR AUC  ROC AUC  F1-Score
Optuna_Tuned_PyTorch  0.1316   0.5963    0.0282

💾 Step 4: 최종 모델 예측 결과를 원본 CSV에 추가하여 저장


전체 데이터 예측:   0%|          | 0/175 [00:00<?, ?it/s]

✅ 모든 데이터의 예측 결과가 '/content/drive/MyDrive/review_helpfulness/PADA/results/s2/hotel/MLP/MLP_hotel_with_DistilBERT_predictions.csv' 파일에 성공적으로 저장되었습니다.

🧹 Step 5: 메모리 정리
✅ 메모리 정리가 완료되었습니다.

🎉 모든 과정이 완료되었습니다!
